## Create TCP Server and client

In [ ]:
import asyncio
from asyncio import StreamReader, StreamWriter, Server

addres = '127.0.0.1'
port = 8888

async def serve_server(server: Server):
    print("Server is running...")
    try:
        await server.serve_forever()
    except asyncio.CancelledError:
        print("Server terminated!")
        
async def stop_server(server):
    await asyncio.sleep(4)
    server.close()

async def server_handle_client(reader: StreamReader, writer: StreamWriter):
    # Wait for message from a client
    data = await reader.read(100)  # Read up to 100 bytes
    message = data.decode('utf8')
    address = writer.get_extra_info('peername')
    print(f"Server: Received {message} from {address}")

    # Send a response to the client
    print("Server: Sending response to client...")
    new_message = "Sever is terminating..."
    new_data = new_message.encode('utf8')
    writer.write(new_data)
    await writer.drain()

    # Close the connection
    writer.close()

async def tcp_echo_client():
    # Connect to the server
    await asyncio.sleep(1)  # Give the server time to start
    reader, writer = await asyncio.open_connection(addres, port)

    # Send a message to the server
    print("Client: Sending message...")
    message = 'Hello World!'
    writer.write(message.encode('utf8'))
    await writer.drain()

    # Receive a response
    data = await reader.read(100)
    print(f"Client: Reveived from Server: {data.decode('utf8')}")

    # Close the connection
    writer.close()
    await writer.wait_closed()

async def main():
     # Start both the server and client concurrently
    server = await asyncio.start_server(server_handle_client, addres, port)
    
    await asyncio.gather(
        serve_server(server),
        tcp_echo_client(),
        stop_server(server)
    )

await main()